In [8]:
import pandas as pd
import numpy as np

import os
path="../../Data/"
import numpy as np
import re
import datetime as dt

In [9]:
keycols=['eid','date_of_attending_assessment_centre_f53_0_0','age_when_attended_assessment_centre_f21003_0_0']

In [10]:
def findcols(df,string):
    return [col for col in df if string in col]

In [11]:
def convert_null(df):
    for col in df.columns:
        mask=(df[col]=='Prefer not to answer')|(df[col]=='nan')
        df[col][mask]=np.NaN
    return df

In [12]:
def onehotencoder(df,cols,excwords,maxrecs=10,mincount=0.8):
    
    #create nulls where unknown for future imputation
    for col in df.columns:
        mask_exc=(df[col].isin(excwords))
        df[col][mask_exc]=np.nan  
    ohe_cols=\
    [col for col in cols if len(df[col].value_counts())<maxrecs
     and df[col].count()/df[col].shape[0]>mincount]
    
    print(r'Total ohe variables = %0.0f ' % (len(ohe_cols)))
        
    df_ohe_cols=df[ohe_cols]
         
    df_out=pd.get_dummies(df_ohe_cols, prefix=df_ohe_cols.columns,
               columns=df_ohe_cols.columns)
       
    df_out['eid']=df['eid'].tolist()
    
    return df_out

In [13]:
excwords=['Prefer not to answer','nan','None of the above']
speccols=['sex_f31','average_total_household_income_before_tax_f738','usual_walking_pace_f924',
 'frequency_of_friendfamily_visits_f1031','drive_faster_than_motorway_speed_limit_f1100',
          'weekly_usage_of_mobile_phone_in_last_3_months_f1120','qualifications_f6138',
         'gender','avgincome','walkspeed','freqfriendfamily','faster_mot_speed','weekly_mobphone_mins',
          'qualif_score','APOE4_Carriers']

In [14]:
'''
create lookup of values by variable so we can map these ordinally where possible
'''


def create_dic(df,maxnum=15):
    return [(col,set(df[col][pd.notnull(df[col])].unique())) for col in df.columns if len(df[col].unique())<=maxnum]

list1=create_dic(ukb_tp0_cols80,maxnum=15)
valslist=pd.DataFrame(list1)
valslist.columns=['column','values']
valslist['merge']=valslist['values'].astype(str)
vallist2=pd.DataFrame(valslist['values'].value_counts()).reset_index()
vallist2.columns=['values','recs']
vallist2['merge']=vallist2['values'].astype(str)
vallistcomp=pd.merge(valslist,vallist2,on='merge',how='left')
vallistcomp.sort_values(by='recs',ascending=False,inplace=True)
#vallistcomp.to_csv(path+'vallistcomp.csv')

NameError: name 'ukb_tp0_cols80' is not defined

In [15]:
ordcols_full=pd.read_csv(path+'vallistcomp_edited.csv')
ordcols=ordcols_full[(ordcols_full['Do']!='Exclude')&(ordcols_full['Do']!='ohe')&pd.notnull(ordcols_full['Do'])][['column','Do']]

In [16]:
cols80_df=pd.read_csv(path+'cols80.csv')

In [17]:
cols80=list(cols80_df['colname'])
excs='source_of_report|first_reported|icd10|icd9|operative_procedures|treatment_speciality|external_ca|patient_recoded|\
hospital_polymorphic|_report|assay_date|device_id'
cols80_exc=[col for col in cols80 if not re.search(excs,col) ]

In [18]:
%%time
ukb_tp0_cols80=pd.read_parquet(path+'ukb_tp0_new.parquet',columns=cols80_exc)
ukb_tp0_cols80=convert_null(ukb_tp0_cols80)

<ipython-input-11-acbc81d1e7af>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][mask]=np.NaN


CPU times: user 27.9 s, sys: 10 s, total: 37.9 s
Wall time: 30.9 s


In [28]:
ukb_tp0_cols80[findcols(ukb_tp0_cols80,'number_of_times_snapbutton_pressed')]

,number_of_times_snapbutton_pressed_f403_0_0,number_of_times_snapbutton_pressed_f403_0_1,number_of_times_snapbutton_pressed_f403_0_2,number_of_times_snapbutton_pressed_f403_0_3,number_of_times_snapbutton_pressed_f403_0_4,number_of_times_snapbutton_pressed_f403_0_5,number_of_times_snapbutton_pressed_f403_0_6,number_of_times_snapbutton_pressed_f403_0_7,number_of_times_snapbutton_pressed_f403_0_8,number_of_times_snapbutton_pressed_f403_0_9,number_of_times_snapbutton_pressed_f403_0_10,number_of_times_snapbutton_pressed_f403_0_11
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
502443,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
502444,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
502445,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
502446,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0


In [26]:
ukb_tp0_cols80['number_of_times_snapbutton_pressed_f403_0_3']

0         0.0
1         0.0
2         1.0
3         0.0
4         1.0
         ... 
502443    0.0
502444    0.0
502445    0.0
502446    1.0
502447    NaN
Name: number_of_times_snapbutton_pressed_f403_0_3, Length: 502448, dtype: float64

In [22]:
ukb_tp0_cols80['number_of_correct_matches_in_round_f398_0_2'].count()

497825

In [67]:
ordvars=[col for col in ordcols['column']]
ctsvars=[col for col in ukb_tp0_cols80.columns if re.search('float',str(ukb_tp0_cols80[col].dtype))
        and not re.search('aliquot|assessment_centre|acquisition_time|main_speciality|date_of_|patient_classi|\
methods_of_discharge|inpatient_record_format|weight_method|_missing_reason|eid',col)]
excvars=list(ordcols_full[(ordcols_full['Do']=='Exclude')]['column'])
ohe_vars=[col for col in ukb_tp0_cols80.columns if col not in list(ordcols['column'])+ctsvars+excvars
         and not re.search('aliquot|assessment_centre|acquisition_time|main_speciality|date_of_|patient_classi|\
methods_of_discharge|inpatient_record_format|weight_method|_missing_reason|eid',col)]

In [52]:
ohe_df=onehotencoder(df=ukb_tp0_cols80[ohe_vars+['eid']],cols=ohe_vars,excwords=excwords)

<ipython-input-43-bbd5783d53e2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][mask_exc]=np.nan


Total ohe variables = 26 


In [64]:
cts_df=ukb_tp0_cols80[ctsvars+['eid']]

In [69]:
df_ord=ukb_tp0_cols80[['eid']+ordvars]

In [72]:
import ast
for i,col in enumerate(ordcols['column']):
    df_ord[col]=ukb_tp0_cols80[col].map(ast.literal_eval(ordcols['Do'].iloc[i]))

<ipython-input-72-7b1f6a3b26ce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ord[col]=ukb_tp0_cols80[col].map(ast.literal_eval(ordcols['Do'].iloc[i]))


In [83]:
df_model=pd.merge(df_ord,cts_df,on='eid',how='left')
df_model=pd.merge(df_model,ohe_df,on='eid',how='left')
df_model=pd.merge(df_model,ukb_tp0_cols80[keycols],on='eid',how='left')

In [86]:
df_model.to_parquet(path+'df_model.parquet')